In [1]:
!pip install numpy tensorflow music21

In [8]:
import numpy as np
import tensorflow as tf
import random
from music21 import converter, instrument, note, chord, stream
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
import pickle
import os

def load_midi(file_path):
    midi = converter.parse(file_path)
    notes = []

    for part in midi.parts:
        for element in part.recurse():
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, sequence_length=100):
    """Prepare input sequences and corresponding outputs for training"""
    # Get unique notes/chords
    unique_notes = sorted(set(notes))
    note_to_int = {note: i for i, note in enumerate(unique_notes)}
    int_to_note = {i: note for i, note in enumerate(unique_notes)}
    
    print(f"Unique notes/chords: {len(unique_notes)}")
    
    # Create input sequences and outputs
    network_input = []
    network_output = []
    
    for i in range(len(notes) - sequence_length):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    
    n_patterns = len(network_input)
    n_vocab = len(unique_notes)
    
    # Reshape input for RNN [samples, time steps, features]
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # Normalize input
    network_input = network_input / float(n_vocab)
    # One-hot encode output
    network_output = to_categorical(network_output)
    
    return network_input, network_output, note_to_int, int_to_note, n_vocab

def create_model(network_input, n_vocab):
    """Create the RNN model architecture"""
    model = Sequential([
        SimpleRNN(256, input_shape=(network_input.shape[1], network_input.shape[2]), 
                  return_sequences=True),
        Dropout(0.3),
        SimpleRNN(256),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(n_vocab, activation='softmax')
    ])
    
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam',
                  metrics=['accuracy'])
    
    return model

def train_model(model, network_input, network_output, epochs=200):
    """Train the model with checkpoints"""
    # Create weights directory if it doesn't exist
    os.makedirs('weights', exist_ok=True)
    
    # Define checkpoint callback
    checkpoint = ModelCheckpoint(
        'weights/weights-improvement-{epoch:02d}-{loss:.4f}.keras',
        monitor='loss',
        verbose=1,
        save_best_only=True,
        mode='min'
    )
    
    callbacks_list = [checkpoint]
    
    # Train the model
    model.fit(network_input, network_output, 
              epochs=epochs, 
              batch_size=64,
              callbacks=callbacks_list,
              verbose=1)
    
    return model

def generate_notes(model, network_input, int_to_note, n_vocab, sequence_length=100, num_notes=500):
    """Generate new music notes using the trained model"""
    # Pick a random sequence from the input as a starting point
    start = np.random.randint(0, len(network_input)-1)
    pattern = network_input[start].flatten().tolist()
    
    prediction_output = []
    
    # Generate notes
    for note_index in range(num_notes):
        # Reshape pattern for prediction
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        
        # Make prediction
        prediction = model.predict(prediction_input, verbose=0)
        
        # Get the index with highest probability
        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        # Update pattern for next prediction
        pattern.append(index / float(n_vocab))
        pattern = pattern[1:len(pattern)]
    
    return prediction_output

def create_midi(prediction_output, filename='generated_music.mid'):
    """Convert predicted notes back to MIDI file"""
    offset = 0
    output_notes = []
    
    for pattern in prediction_output:
        # Check if pattern is a chord or a note
        if ('.' in pattern) or pattern.isdigit():
            # It's a chord
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            # It's a note
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        
        # Increase offset each iteration so notes don't stack
        offset += 0.5
    
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=filename)
    print(f"Generated MIDI saved as: {filename}")

def save_training_data(note_to_int, int_to_note, filename='training_data.pkl'):
    """Save training mappings for later use"""
    with open(filename, 'wb') as f:
        pickle.dump({'note_to_int': note_to_int, 'int_to_note': int_to_note}, f)

def load_training_data(filename='training_data.pkl'):
    """Load training mappings"""
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data['note_to_int'], data['int_to_note']

# Main execution
if __name__ == "__main__":
    # Load dataset
    file_path = "ALeagueOfTheirOwn.mid"  # The Music File
    
    try:
        notes = load_midi(file_path)
        print(f"Total notes/chords: {len(notes)}")
        
        if len(notes) < 100:
            print("Warning: Not enough notes for effective training. Consider using a longer MIDI file.")
        
        # Prepare sequences
        sequence_length = 100
        network_input, network_output, note_to_int, int_to_note, n_vocab = prepare_sequences(notes, sequence_length)
        
        print(f"Network input shape: {network_input.shape}")
        print(f"Network output shape: {network_output.shape}")
        
        # Save training data mappings
        save_training_data(note_to_int, int_to_note)
        
        # Create model
        model = create_model(network_input, n_vocab)
        print("Model created successfully!")
        print(model.summary())
        
        # Train model
        print("Starting training...")
        try:
            model = train_model(model, network_input, network_output, epochs=50)  # Reduced epochs for testing
        except Exception as train_error:
            print(f"Training error: {train_error}")
            print("Continuing with untrained model for demonstration...")
        
        # Save the final model
        try:
            model.save('final_music_model.keras')
            print("Model saved as 'final_music_model.keras'")
        except Exception as save_error:
            print(f"Error saving model: {save_error}")
            print("Continuing without saving...")
        
        # Generate new music
        print("Generating new music...")
        prediction_output = generate_notes(model, network_input, int_to_note, n_vocab, sequence_length, 500)
        
        # Create MIDI file
        create_midi(prediction_output, 'generated_music.mid')
        print("Music generation complete!")
        
    except FileNotFoundError:
        print(f"Error: Could not find MIDI file '{file_path}'")
        print("Please make sure the MIDI file exists in the current directory.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Make sure you have music21 installed: pip install music21")

# Optional: Function to generate music from a saved model
def generate_from_saved_model(model_path='final_music_model.keras', 
                             training_data_path='training_data.pkl',
                             original_midi_path="ALeagueOfTheirOwn.mid"):
    """Generate music from a previously saved model"""
    try:
        # Load the trained model
        model = tf.keras.models.load_model(model_path)
        
        # Load training data mappings
        note_to_int, int_to_note = load_training_data(training_data_path)
        
        # Load original notes to create input sequences
        notes = load_midi(original_midi_path)
        network_input, _, _, _, n_vocab = prepare_sequences(notes, 100)
        
        # Generate new music
        prediction_output = generate_notes(model, network_input, int_to_note, n_vocab, 100, 500)
        
        # Create MIDI file
        create_midi(prediction_output, 'new_generated_music.mid')
        print("New music generated from saved model!")
        
    except Exception as e:
        print(f"Error generating from saved model: {str(e)}")

# Uncomment the line below to generate music from a saved model
# generate_from_saved_model()

Total notes/chords: 2045
Unique notes/chords: 84
Network input shape: (1945, 100, 1)
Network output shape: (1945, 84)
Model created successfully!


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_2 (SimpleRNN)        │ (None, 100, 256)       │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 84)             │        21,588 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 284,756 (1.09 MB)

 Trainable params: 284,756 (1.09 MB)

 Non-trainable params: 0 (0.00 B)

None
Starting training...
Epoch 1/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.0529 - loss: 4.2748
Epoch 1: loss improved from inf to 4.25476, saving model to weights/weights-improvement-01-4.2548.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 179ms/step - accuracy: 0.0528 - loss: 4.2741
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.0532 - loss: 4.0699
Epoch 2: loss improved from 4.25476 to 4.08706, saving model to weights/weights-improvement-02-4.0871.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 178ms/step - accuracy: 0.0531 - loss: 4.0704
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - accuracy: 0.0365 - loss: 4.0503
Epoch 3: loss improved from 4.08706 to 4.04236, saving model to weights/weights-improvement-03-4.0424.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 173ms/step - accuracy: 0.0368 - loss: 4.0501
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.0553 - loss: 4.0745
Epoch 4: loss improved from 4.04236 to 4.01377, saving model to weights/weights-impr

In [9]:
generate_from_saved_model()

Unique notes/chords: 84
Generated MIDI saved as: new_generated_music.mid
New music generated from saved model!
